In [1]:
import math


# --- 1. Random Number Generator (LCG) ---

class LCG:
    def __init__(self, a, c, m, seed):
        self.a = a
        self.c = c
        self.m = m
        self.z = seed  # Current Z_i value

    def rand(self):
        """Calculates the next Z value and returns an RN in the range [0, 1)."""
        self.z = (self.a * self.z + self.c) % self.m
        return self.z / self.m

# --- 2. Entity Class ---

class Customer:
    def __init__(self, customer_id, arrival_time):
        self.id = customer_id
        self.arrival_time = arrival_time
        self.wait_time = 0.0
        self.status = None  # 'lucky' or 'unlucky'
        self.tickets_bought = 0

# --- 3. Simulation Model ---
class CinemaSimulation:
    def __init__(self, lcg_params):
        self.rng = LCG(**lcg_params)
        self.clock = 0.0
        self.fel = []  # Future Event List (Managed manually)
        self.server_status = 0  # 0=IDLE, 1=BUSY
        self.queue = []         # Customer queue (FIFO)
        self.tickets_sold = 0
        self.ticket_capacity = 250
        self.current_customer_in_service = None
        self.next_customer_id = 1

        # Statistics
        self.last_event_time = 0.0
        self.area_under_queue_length = 0.0
        self.server_busy_time = 0.0
        self.total_wait_lucky = 0.0
        self.num_waited_lucky = 0
        self.total_wait_unlucky = 0.0
        self.num_waited_unlucky = 0
        self.verification_log = True

    # --- Event List Management (Manual) ---

    def schedule_event(self, time, event_type, entity=None):
        """Adds an event to the FEL and ensures it remains sorted by time."""
        new_event = (time, event_type, entity)
        for i in range(len(self.fel)):
            if new_event[0] < self.fel[i][0]:
                self.fel.insert(i, new_event)
                return
        self.fel.append(new_event)

    def get_next_event(self):
        """Pulls the earliest event from the FEL (beginning of the list)."""
        if not self.fel:
            return None
        return self.fel.pop(0)

    # --- Random Variable Generators (Manual) ---
    def get_interarrival_time(self):
        """Generates IAT using Exp(mean=8)."""
        r = self.rng.rand()
        return -8.0 * math.log(r) #

    # --- CORRECTION 1: get_service_details ---
    # Updated according to the request for RN consumption order (Tix -> Pay -> ST).
    def get_service_details(self):
        """Determines service time and number of tickets."""

        # 1. Number of Tickets (Requested order #1)
        r_tix = self.rng.rand()
        if r_tix <= 0.30:       # 30% 1 ticket
            num_tickets = 1
        elif r_tix <= 0.70:     # 40% 2 tickets
            num_tickets = 2
        elif r_tix <= 0.90:     # 20% 3 tickets
            num_tickets = 3
        else:                   # 10% 4 tickets
            num_tickets = 4

        # 2. Payment Method (Requested order #2)
        r_pay = self.rng.rand()
        if r_pay <= 0.25:  # 25% Cash
            # 3. Service Time (Cash) (Requested order #3)
            r_st = self.rng.rand()
            service_time = 2.0 + (7.0 - 2.0) * r_st  # U(2, 7)
        else:  # 75% Card
            # 3. Service Time (Card) (Requested order #3)
            r_st = self.rng.rand()
            service_time = 2.0 + (4.0 - 2.0) * r_st  # U(2, 4)

        return service_time, num_tickets

    # --- Statistics Update ---

    def update_stats(self):
        """Updates time-weighted statistics."""
        time_delta = self.clock - self.last_event_time
        if time_delta > 0:
            self.area_under_queue_length += len(self.queue) * time_delta
            self.server_busy_time += self.server_status * time_delta
        self.last_event_time = self.clock

    # --- Event Procedures ---

    def handle_init(self):
        """Initializes the simulation, schedules the first arrival."""
        if self.verification_log:
            self.log_state("Init", None)
        # Schedule only the first arrival
        first_iat = self.get_interarrival_time()
        self.schedule_event(first_iat, "CUSTOMER_ARRIVAL")

    # --- CORRECTION 2: handle_arrival ---
    # IAT consumption logic corrected.
    # The call to 'get_interarrival_time()' has been moved to AFTER the current customer
    # finishes service or queuing.
    def handle_arrival(self, event):
        """Handles a customer arrival."""
        self.update_stats()

        customer = Customer(self.next_customer_id, self.clock)
        self.next_customer_id += 1

        if self.verification_log:
            self.log_state(f"Arr (C{customer.id})", None)

        # Check server status
        if self.server_status == 0:  # Idle
            # Start service FIRST (This consumes Tix, Pay, ST RNs)
            self.start_service(customer)
        else:  # Busy
            # Add to queue (This consumes no RNs)
            self.queue.append(customer)

        # NOW schedule the next arrival (consumes IAT RN).
        # This uses the correct RN (R5) after C1
        iat = self.get_interarrival_time()
        next_arrival_time = self.clock + iat
        if next_arrival_time < 510.0:  # If before 18:30
            self.schedule_event(next_arrival_time, "CUSTOMER_ARRIVAL")

    def start_service(self, customer):
        """Starts a customer's service (secondary event)."""
        self.server_status = 1
        customer.wait_time = self.clock - customer.arrival_time

        # GET service time and tickets (RNs are consumed here)
        st, tix = self.get_service_details()

        # Check ticket availability
        if (self.tickets_sold + tix) <= self.ticket_capacity:
            customer.status = 'lucky'
            customer.tickets_bought = tix
            self.tickets_sold += tix
        else:
            customer.status = 'unlucky'
            customer.tickets_bought = 0

        self.current_customer_in_service = customer

        # Schedule service completion event
        completion_time = self.clock + st
        self.schedule_event(completion_time, "SERVICE_COMPLETION", customer)

    def handle_service_completion(self, event):
        """Handles the completion of a customer's service."""
        self.update_stats()

        customer = event[2]  # Customer associated with the event
        if self.verification_log:
            self.log_state(f"Srv_Comp (C{customer.id})", customer)

        # Collect statistics
        if customer.status == 'lucky':
            self.total_wait_lucky += customer.wait_time
            self.num_waited_lucky += 1
        else:
            self.total_wait_unlucky += customer.wait_time
            self.num_waited_unlucky += 1

        self.current_customer_in_service = None

        # Check the queue
        if len(self.queue) > 0:
            next_customer = self.queue.pop(0)
            self.start_service(next_customer)
        else:
            self.server_status = 0  # Set server to idle

    def handle_stop_sales(self, event):
        """Ends the simulation (and the day)."""
        self.update_stats()
        if self.verification_log:
            self.log_state("Stop_Sales", None)

        # Everyone remaining in the queue becomes "unlucky"
        for customer in self.queue:
            wait = self.clock - customer.arrival_time
            self.total_wait_unlucky += wait
            self.num_waited_unlucky += 1
        self.queue = []
        self.fel = [] # Stop the loop

    # --- Verification and Reporting ---

    def log_state(self, event_name, entity):
        """Generates output similar to the manual simulation table for Task 4."""

        # Show a summary of the FEL
        fel_summary = []
        # Note: Using 'self.fel' instead of 'sorted(self.fel)' might be
        # more efficient as you are already inserting in a sorted manner,
        # but this version is also correct.
        sorted_fel = sorted(self.fel)
        for i, (time, ev_type, ent) in enumerate(sorted_fel):
            if i >= 5:
                fel_summary.append("...")
                break
            ent_id = f"C{ent.id}" if ent else "-"
            fel_summary.append(f"({time:.2f}, {ev_type[:3]}, {ent_id})")

        print(f"Clock: {self.clock:6.2f} | "
              f"Event: {event_name:<18} | "
              f"B(t): {self.server_status} | "
              f"LQ(t): {len(self.queue):2} | "
              f"Tix Sold: {self.tickets_sold:3} | "
              f"FEL: {' '.join(fel_summary)}")

    def report_stats(self, stop_time):
        """Calculates and prints end-of-simulation statistics."""
        print("\n--- Simulation Report ---")

        # Perform the last statistics update before reporting
        self.update_stats()

        avg_wait_lucky = (self.total_wait_lucky / self.num_waited_lucky) if self.num_waited_lucky > 0 else 0
        avg_wait_unlucky = (self.total_wait_unlucky / self.num_waited_unlucky) if self.num_waited_unlucky > 0 else 0
        avg_queue_len = self.area_under_queue_length / stop_time
        utilization = self.server_busy_time / stop_time

        print(f"Simulation Time:    {stop_time:.2f} minutes")
        print(f"Tickets Sold:        {self.tickets_sold} / {self.ticket_capacity}")
        print("-" * 30)
        print(f"Server Utilization:       {utilization: .4f}")
        print(f"Avg Queue Length:    {avg_queue_len: .4f} customers")
        print(f"Avg Wait (Lucky): {avg_wait_lucky: .4f} min ({self.num_waited_lucky} people)")
        print(f"Avg Wait (Unlucky):{avg_wait_unlucky: .4f} min ({self.num_waited_unlucky} people)")

    # --- Main Run Loop ---
    def run(self, stop_time):
        """Runs the simulation until 'stop_time'."""
        self.schedule_event(0.0, "INIT")
        self.schedule_event(stop_time, "STOP_SALES")

        while len(self.fel) > 0:
            event = self.get_next_event()
            if event is None:
                break

            time, event_type, entity = event

            # Do not process an event that exceeds 30.0 (except STOP_SALES)
            # This guarantees stopping at 30.0
            if time > stop_time and event_type != "STOP_SALES":
                continue # Skip this event

            self.clock = time

            if event_type == "INIT":
                self.handle_init()
            elif event_type == "CUSTOMER_ARRIVAL":
                self.handle_arrival(event)
            elif event_type == "SERVICE_COMPLETION":
                self.handle_service_completion(event)
            elif event_type == "STOP_SALES":
                # Set clock exactly to stop_time
                self.clock = stop_time
                self.handle_stop_sales(event)
                break  # End simulation

        # Reporting
        self.report_stats(stop_time)

# --- Task 4: Verification Run (30 Minutes) ---
print("--- Task 4: 30 Minute Verification Run (CORRECTED) ---")
lcg_params = {'a': 29, 'c': 3, 'm': 1289, 'seed': 541}

sim_verify = CinemaSimulation(lcg_params)
sim_verify.run(stop_time=30.0)

--- Task 4: 30 Minute Verification Run (CORRECTED) ---
Clock:   0.00 | Event: Init               | B(t): 0 | LQ(t):  0 | Tix Sold:   0 | FEL: (30.00, STO, -)
Clock:  14.00 | Event: Arr (C1)           | B(t): 0 | LQ(t):  0 | Tix Sold:   0 | FEL: (30.00, STO, -)
Clock:  16.26 | Event: Arr (C2)           | B(t): 1 | LQ(t):  0 | Tix Sold:   1 | FEL: (17.51, SER, C1) (30.00, STO, -)
Clock:  17.37 | Event: Arr (C3)           | B(t): 1 | LQ(t):  1 | Tix Sold:   1 | FEL: (17.51, SER, C1) (30.00, STO, -)
Clock:  17.51 | Event: Srv_Comp (C1)      | B(t): 1 | LQ(t):  2 | Tix Sold:   1 | FEL: (28.62, CUS, -) (30.00, STO, -)
Clock:  19.62 | Event: Srv_Comp (C2)      | B(t): 1 | LQ(t):  1 | Tix Sold:   2 | FEL: (28.62, CUS, -) (30.00, STO, -)
Clock:  23.24 | Event: Srv_Comp (C3)      | B(t): 1 | LQ(t):  0 | Tix Sold:   4 | FEL: (28.62, CUS, -) (30.00, STO, -)
Clock:  28.62 | Event: Arr (C4)           | B(t): 0 | LQ(t):  0 | Tix Sold:   4 | FEL: (30.00, STO, -)
Clock:  30.00 | Event: Stop_Sales      

In [2]:
import math
# heapq or numpy libraries were NOT used
# --- 1. Random Number Generator (LCG) ---# Implements the LCG specified in the assignment
class LCG:
    def __init__(self, a, c, m, seed):
        self.a = a
        self.c = c
        self.m = m
        self.z = seed

    def rand(self):
        """Calculates the next Z value and returns an RN in the range [0, 1)."""
        self.z = (self.a * self.z + self.c) % self.m
        return self.z / self.m

# --- 2. Entity Class ---
# (No changes)
class Customer:
    def __init__(self, customer_id, arrival_time):
        self.id = customer_id
        self.arrival_time = arrival_time
        self.wait_time = 0.0
        self.status = None
        self.tickets_bought = 0

# --- NEW HELPER FUNCTION: Time Formatter ---
def format_time(decimal_minutes):
    """Converts decimal minutes (Clock) to HH:MM:SS format starting from 10:00."""

    # Find the total seconds to add on top of 10:00
    # e.g., 17.51 min -> 17.51 * 60 = 1050.6 seconds -> 1051 seconds
    total_seconds_offset = round(decimal_minutes * 60)

    # Start time (10:00:00) in seconds
    start_time_seconds = 10 * 3600

    # New total seconds
    current_total_seconds = start_time_seconds + total_seconds_offset

    # Convert to HH:MM:SS
    h = current_total_seconds // 3600
    m = (current_total_seconds % 3600) // 60
    s = (current_total_seconds % 3600) % 60

    # Format to appear like :05, :09
    return f"{h:02}:{m:02}:{s:02}"

# --- 3. Simulation Model ---
class CinemaSimulation:
    def __init__(self, lcg_params):
        self.rng = LCG(**lcg_params)
        self.clock = 0.0
        self.fel = []  # Future Event List (Managed manually)
        self.server_status = 0
        self.queue = []
        self.tickets_sold = 0
        self.ticket_capacity = 250
        self.current_customer_in_service = None
        self.next_customer_id = 1

        # Statistics
        self.last_event_time = 0.0
        self.area_under_queue_length = 0.0
        self.server_busy_time = 0.0
        self.total_wait_lucky = 0.0
        self.num_waited_lucky = 0
        self.total_wait_unlucky = 0.0
        self.num_waited_unlucky = 0
        self.verification_log = True

    # --- Event List Management (Manual) ---
    def schedule_event(self, time, event_type, entity=None):
        """Adds an event to the FEL and ensures it remains sorted by time."""
        new_event = (time, event_type, entity)
        for i in range(len(self.fel)):
            if new_event[0] < self.fel[i][0]:
                self.fel.insert(i, new_event)
                return
        self.fel.append(new_event)

    def get_next_event(self):
        """Pulls the earliest event from the FEL (beginning of the list)."""
        if not self.fel:
            return None
        return self.fel.pop(0)

    # --- Random Variable Generators (Manual) ---
    def get_interarrival_time(self):
        """Generates IAT using Exp(mean=8)."""
        r = self.rng.rand()
        return -8.0 * math.log(r)

    # RN Consumption Order: Tix -> Pay -> ST (Corrected as requested)
    def get_service_details(self):
        """Determines service time and number of tickets."""

        # 1. Number of Tickets
        r_tix = self.rng.rand()
        if r_tix <= 0.30:       # 30% 1 ticket
            num_tickets = 1
        elif r_tix <= 0.70:     # 40% 2 tickets
            num_tickets = 2
        elif r_tix <= 0.90:     # 20% 3 tickets
            num_tickets = 3
        else:                   # 10% 4 tickets
            num_tickets = 4

        # 2. Payment Method
        r_pay = self.rng.rand()
        if r_pay <= 0.25:  # 25% Cash
            # 3. Service Time (Cash)
            r_st = self.rng.rand()
            service_time = 2.0 + (7.0 - 2.0) * r_st  # U(2, 7)
        else:  # 75% Card
            # 3. Service Time (Card)
            r_st = self.rng.rand()
            service_time = 2.0 + (4.0 - 2.0) * r_st  # U(2, 4)

        return service_time, num_tickets

    # --- Statistics Update ---
    def update_stats(self):
        """Updates time-weighted statistics."""
        time_delta = self.clock - self.last_event_time
        if time_delta > 0:
            self.area_under_queue_length += len(self.queue) * time_delta
            self.server_busy_time += self.server_status * time_delta
        self.last_event_time = self.clock

    # --- Event Procedures ---
    def handle_init(self):
        """Initializes the simulation, schedules the first arrival."""
        if self.verification_log:
            self.log_state("Init", None)
        # Schedule only the first arrival (IAT RN#1)
        first_iat = self.get_interarrival_time()
        self.schedule_event(first_iat, "CUSTOMER_ARRIVAL")

    # RN Consumption Order Corrected
    def handle_arrival(self, event):
        """Handles a customer arrival."""
        self.update_stats()
        customer = Customer(self.next_customer_id, self.clock)
        self.next_customer_id += 1

        if self.verification_log:
            self.log_state(f"Arr (C{customer.id})", None)

        # Check server status
        if self.server_status == 0:  # Idle
            # Start service FIRST (Tix RN#2, Pay RN#3, ST RN#4 are consumed)
            self.start_service(customer)
        else:  # Busy
            # Add to queue (No RNs are consumed)
            self.queue.append(customer)

        # THEN schedule the next arrival (IAT RN#5 is consumed)
        iat = self.get_interarrival_time()
        next_arrival_time = self.clock + iat
        if next_arrival_time < 510.0:  # If before 18:30
            self.schedule_event(next_arrival_time, "CUSTOMER_ARRIVAL")

    def start_service(self, customer):
        """Starts a customer's service (secondary event)."""
        self.server_status = 1
        customer.wait_time = self.clock - customer.arrival_time
        st, tix = self.get_service_details() # RNs are consumed here

        if (self.tickets_sold + tix) <= self.ticket_capacity:
            customer.status = 'lucky'
            customer.tickets_bought = tix
            self.tickets_sold += tix
        else:
            customer.status = 'unlucky'
            customer.tickets_bought = 0

        self.current_customer_in_service = customer

        # LOGIC STILL USES DECIMAL MINUTES
        completion_time = self.clock + st # e.g., 14.00 + 3.51 = 17.51
        self.schedule_event(completion_time, "SERVICE_COMPLETION", customer)

    def handle_service_completion(self, event):
        """Handles the completion of a customer's service."""
        self.update_stats()
        customer = event[2]

        if self.verification_log:
            self.log_state(f"Srv_Comp (C{customer.id})", customer)

        if customer.status == 'lucky':
            self.total_wait_lucky += customer.wait_time
            self.num_waited_lucky += 1
        else:
            self.total_wait_unlucky += customer.wait_time
            self.num_waited_unlucky += 1

        self.current_customer_in_service = None

        if len(self.queue) > 0:
            next_customer = self.queue.pop(0)
            self.start_service(next_customer)
        else:
            self.server_status = 0

    def handle_stop_sales(self, event):
        """Ends the simulation (and the day)."""
        self.update_stats()
        if self.verification_log:
            self.log_state("Stop_Sales", None)

        for customer in self.queue:
            wait = self.clock - customer.arrival_time
            self.total_wait_unlucky += wait
            self.num_waited_unlucky += 1
        self.queue = []
        self.fel = []

    # --- REVISED SECTION: log_state ---
    def log_state(self, event_name, entity):
        """Generates output in HH:MM:SS format for Task 4."""

        fel_summary = []
        sorted_fel = sorted(self.fel)
        for i, (time, ev_type, ent) in enumerate(sorted_fel):
            if i >= 5:
                fel_summary.append("...")
                break
            ent_id = f"C{ent.id}" if ent else "-"
            # Format the time within the FEL as well
            fel_summary.append(f"({format_time(time)}, {ev_type[:3]}, {ent_id})")

        # Output format UPDATED to HH:MM:SS
        print(f"Time: {format_time(self.clock)} | "
              f"Event: {event_name:<18} | "
              f"B(t): {self.server_status} | "
              f"LQ(t): {len(self.queue):2} | "
              f"Tix Sold: {self.tickets_sold:3} | "
              f"FEL: {' '.join(fel_summary)}")

    def report_stats(self, stop_time):
        """Calculates and prints end-of-simulation statistics."""
        print("\n--- Simulation Report ---")
        self.update_stats()

        avg_wait_lucky = (self.total_wait_lucky / self.num_waited_lucky) if self.num_waited_lucky > 0 else 0
        avg_wait_unlucky = (self.total_wait_unlucky / self.num_waited_unlucky) if self.num_waited_unlucky > 0 else 0
        avg_queue_len = self.area_under_queue_length / stop_time
        utilization = self.server_busy_time / stop_time

        # Reporting is still in decimal minutes (standard statistics)
        print(f"Simulation Time:    {stop_time:.2f} minutes")
        print(f"Tickets Sold:        {self.tickets_sold} / {self.ticket_capacity}")
        print("-" * 30)
        print(f"Server Utilization:       {utilization: .4f}")
        print(f"Avg Queue Length:    {avg_queue_len: .4f} customers")
        print(f"Avg Wait (Lucky): {avg_wait_lucky: .4f} min ({self.num_waited_lucky} people)")
        print(f"Avg Wait (Unlucky):{avg_wait_unlucky: .4f} min ({self.num_waited_unlucky} people)")

    # --- Main Run Loop ---
    def run(self, stop_time):
        """Runs the simulation until 'stop_time'."""
        self.schedule_event(0.0, "INIT")
        self.schedule_event(stop_time, "STOP_SALES")

        while len(self.fel) > 0:
            event = self.get_next_event()
            if event is None:
                break

            time, event_type, entity = event

            # Process events exceeding stop_time (except STOP_SALES)
            if time > stop_time and event_type != "STOP_SALES":
                continue

            self.clock = time

            if event_type == "INIT": self.handle_init()
            elif event_type == "CUSTOMER_ARRIVAL": self.handle_arrival(event)
            elif event_type == "SERVICE_COMPLETION": self.handle_service_completion(event)
            elif event_type == "STOP_SALES":
                self.clock = stop_time # Set clock exactly to 30.0
                self.handle_stop_sales(event)
                break

        # Reporting
        self.report_stats(stop_time)

# --- Task 4: Verification Run (30 Minutes) ---
print("--- Task 4: 30 Minute Verification Run (HH:MM:SS Format) ---")
lcg_params = {'a': 29, 'c': 3, 'm': 1289, 'seed': 541}

sim_verify = CinemaSimulation(lcg_params)
sim_verify.run(stop_time=30.0)

--- Task 4: 30 Minute Verification Run (HH:MM:SS Format) ---
Time: 10:00:00 | Event: Init               | B(t): 0 | LQ(t):  0 | Tix Sold:   0 | FEL: (10:30:00, STO, -)
Time: 10:14:00 | Event: Arr (C1)           | B(t): 0 | LQ(t):  0 | Tix Sold:   0 | FEL: (10:30:00, STO, -)
Time: 10:16:15 | Event: Arr (C2)           | B(t): 1 | LQ(t):  0 | Tix Sold:   1 | FEL: (10:17:31, SER, C1) (10:30:00, STO, -)
Time: 10:17:22 | Event: Arr (C3)           | B(t): 1 | LQ(t):  1 | Tix Sold:   1 | FEL: (10:17:31, SER, C1) (10:30:00, STO, -)
Time: 10:17:31 | Event: Srv_Comp (C1)      | B(t): 1 | LQ(t):  2 | Tix Sold:   1 | FEL: (10:28:37, CUS, -) (10:30:00, STO, -)
Time: 10:19:37 | Event: Srv_Comp (C2)      | B(t): 1 | LQ(t):  1 | Tix Sold:   2 | FEL: (10:28:37, CUS, -) (10:30:00, STO, -)
Time: 10:23:14 | Event: Srv_Comp (C3)      | B(t): 1 | LQ(t):  0 | Tix Sold:   4 | FEL: (10:28:37, CUS, -) (10:30:00, STO, -)
Time: 10:28:37 | Event: Arr (C4)           | B(t): 0 | LQ(t):  0 | Tix Sold:   4 | FEL: (10:3

In [3]:
import math
# import heapq
import numpy as np  # for calculating 7-day averages

# --- 1. Random Number Generator (LCG) ---
# (No changes)
class LCG:
    def __init__(self, a, c, m, seed):
        self.a = a
        self.c = c
        self.m = m
        self.z = seed

    def rand(self):
        self.z = (self.a * self.z + self.c) % self.m
        return self.z / self.m

# --- 2. Entity Class ---
# (No changes)
class Customer:
    def __init__(self, customer_id, arrival_time):
        self.id = customer_id
        self.arrival_time = arrival_time
        self.wait_time = 0.0
        self.status = None
        self.tickets_bought = 0

# --- 3. Simulation Model (Updated for Task 5) ---
class CinemaSimulation:
    # __init__ now takes the LCG object directly instead of LCG parameters
    def __init__(self, lcg_object):
        self.rng = lcg_object  # Use the shared LCG object

        # Simulation Clock and Event List
        self.clock = 0.0
        self.fel = [] # Future Event List (Managed manually)

        # State Variables
        self.server_status = 0
        self.queue = []
        self.tickets_sold = 0
        self.ticket_capacity = 250
        self.current_customer_in_service = None
        self.next_customer_id = 1

        # Statistics Accumulators
        self.last_event_time = 0.0
        self.area_under_queue_length = 0.0
        self.server_busy_time = 0.0
        self.total_wait_lucky = 0.0
        self.num_waited_lucky = 0
        self.total_wait_unlucky = 0.0
        self.num_waited_unlucky = 0

        # Verification logging is turned off by default
        self.verification_log = False

    # --- Event List Management (Manual) ---
    def schedule_event(self, time, event_type, entity=None):
        """Adds an event to the FEL and ensures it remains sorted by time."""
        new_event = (time, event_type, entity)
        # Insert the event at the correct position (similar to insertion sort)
        for i in range(len(self.fel)):
            if new_event[0] < self.fel[i][0]:
                self.fel.insert(i, new_event)
                return
        self.fel.append(new_event) # Add to the end of the list

    def get_next_event(self):
        """Pulls the earliest event from the FEL (beginning of the list)."""
        if not self.fel:
            return None
        return self.fel.pop(0) # Pull from the beginning of the list

    # --- Random Variable Generators (No changes) ---
    def get_interarrival_time(self):
        r = self.rng.rand()
        return -8.0 * math.log(r)

    def get_service_details(self):
        r_pay = self.rng.rand()
        if r_pay <= 0.25:
            r_st = self.rng.rand()
            service_time = 2.0 + 5.0 * r_st  # U(2, 7)
        else:
            r_st = self.rng.rand()
            service_time = 2.0 + 2.0 * r_st  # U(2, 4)

        r_tix = self.rng.rand()
        if r_tix <= 0.30: num_tickets = 1
        elif r_tix <= 0.70: num_tickets = 2
        elif r_tix <= 0.90: num_tickets = 3
        else: num_tickets = 4

        return service_time, num_tickets

    # --- Statistics Update (No changes) ---
    def update_stats(self):
        time_delta = self.clock - self.last_event_time
        if time_delta > 0:
            self.area_under_queue_length += len(self.queue) * time_delta
            self.server_busy_time += self.server_status * time_delta
        self.last_event_time = self.clock

    # --- Event Procedures (No changes) ---
    def handle_init(self):
        if self.verification_log: self.log_state("Init", None)
        first_iat = self.get_interarrival_time()
        self.schedule_event(first_iat, "CUSTOMER_ARRIVAL")

    def handle_arrival(self, event):
        self.update_stats()
        customer = Customer(self.next_customer_id, self.clock)
        self.next_customer_id += 1
        if self.verification_log: self.log_state(f"Arr (C{customer.id})", None)

        iat = self.get_interarrival_time()
        next_arrival_time = self.clock + iat
        if next_arrival_time < 510.0:
            self.schedule_event(next_arrival_time, "CUSTOMER_ARRIVAL")

        if self.server_status == 0:
            self.start_service(customer)
        else:
            self.queue.append(customer)

    def start_service(self, customer):
        self.server_status = 1
        customer.wait_time = self.clock - customer.arrival_time
        st, tix = self.get_service_details()

        if (self.tickets_sold + tix) <= self.ticket_capacity:
            customer.status = 'lucky'
            customer.tickets_bought = tix
            self.tickets_sold += tix
        else:
            customer.status = 'unlucky'
            customer.tickets_bought = 0

        self.current_customer_in_service = customer
        completion_time = self.clock + st
        self.schedule_event(completion_time, "SERVICE_COMPLETION", customer)

    def handle_service_completion(self, event):
        self.update_stats()
        customer = event[2]
        if self.verification_log: self.log_state(f"Srv_Comp (C{customer.id})", customer)

        if customer.status == 'lucky':
            self.total_wait_lucky += customer.wait_time
            self.num_waited_lucky += 1
        else:
            self.total_wait_unlucky += customer.wait_time
            self.num_waited_unlucky += 1

        self.current_customer_in_service = None

        if len(self.queue) > 0:
            next_customer = self.queue.pop(0)
            self.start_service(next_customer)
        else:
            self.server_status = 0

    def handle_stop_sales(self, event):
        self.update_stats()
        if self.verification_log: self.log_state("Stop_Sales", None)

        for customer in self.queue:
            wait = self.clock - customer.arrival_time
            self.total_wait_unlucky += wait
            self.num_waited_unlucky += 1
        self.queue = []
        self.fel = [] # Stop the loop

    def log_state(self, event_name, entity):
        # (This function will not be used in Task 5 but remains for Task 4)
        if self.clock > 30.0:
            self.verification_log = False
            return
        fel_summary = []
        # sorted_fel = sorted(self.fel)
        for i, (time, ev_type, ent) in enumerate(self.fel):
            if i >= 3: fel_summary.append("..."); break
            ent_id = f"C{ent.id}" if ent else "-"
            fel_summary.append(f"({time:.2f}, {ev_type[:3]}, {ent_id})")
        print(f"Clock: {self.clock:6.2f} | Event: {event_name:<18} | B(t): {self.server_status} | LQ(t): {len(self.queue):2} | Tix Sold: {self.tickets_sold:3} | FEL: {' '.join(fel_summary)}")

    # --- `report_stats` method UPDATED to `get_final_stats` ---
    def get_final_stats(self, stop_time):
        """Calculates and returns end-of-day statistics."""
        # Ensure simulation runs exactly until stop_time
        if self.clock < stop_time:
            self.clock = stop_time

        self.update_stats() # Update statistics for the last time interval

        avg_wait_lucky = (self.total_wait_lucky / self.num_waited_lucky) if self.num_waited_lucky > 0 else 0.0
        avg_wait_unlucky = (self.total_wait_unlucky / self.num_waited_unlucky) if self.num_waited_unlucky > 0 else 0.0
        avg_queue_len = self.area_under_queue_length / stop_time
        utilization = self.server_busy_time / stop_time

        return {
            "tickets_sold": self.tickets_sold,
            "lucky_customers": self.num_waited_lucky,
            "unlucky_customers": self.num_waited_unlucky,
            "utilization": utilization,
            "avg_queue_len": avg_queue_len,
            "avg_wait_lucky": avg_wait_lucky,
            "avg_wait_unlucky": avg_wait_unlucky,
        }

    # --- Main Run Loop (No changes) ---
    def run(self, stop_time):
        self.schedule_event(0.0, "INIT")
        self.schedule_event(stop_time, "STOP_SALES")

        while len(self.fel) > 0:
            event = self.get_next_event()
            if event is None: # If FEL is empty
                break
            time, event_type, entity = event
            self.clock = time

            if event_type == "INIT": self.handle_init()
            elif event_type == "CUSTOMER_ARRIVAL": self.handle_arrival(event)
            elif event_type == "SERVICE_COMPLETION": self.handle_service_completion(event)
            elif event_type == "STOP_SALES":
                self.handle_stop_sales(event)
                break

# --- Task 5: 7-Day Simulation (4 Replications) ---

print("--- Task 5: 7-Day Simulation (4 Replications) Run ---")

# LCG Parameters
LCG_A = 29
LCG_C = 3
LCG_M = 1289

# 4 different seeds
SEEDS = [541, 123, 789, 1000]
NUM_DAYS = 7  # A 7-day week
SIM_STOP_TIME = 510.0 # 10:00 to 18:30 (8.5 hours * 60 min)

all_replications_data = []

# 4 REPLICATION LOOP
for rep_idx, seed in enumerate(SEEDS):

    # Create a SINGLE LCG object for the entire 7 days
    lcg_for_replication = LCG(LCG_A, LCG_C, LCG_M, seed)

    daily_stats_list = []

    # 7 DAY INNER LOOP
    for day in range(NUM_DAYS):
        # Start a new simulation for each day using the same LCG object
        sim_day = CinemaSimulation(lcg_for_replication)
        sim_day.run(stop_time=SIM_STOP_TIME)

        # Get the day's statistics
        stats = sim_day.get_final_stats(SIM_STOP_TIME)
        daily_stats_list.append(stats)

    # Calculate the 7-day average
    avg_replication_stats = {
        "replication": rep_idx + 1,
        "seed": seed,
        "avg_tickets_sold": np.mean([d["tickets_sold"] for d in daily_stats_list]),
        "avg_unlucky": np.mean([d["unlucky_customers"] for d in daily_stats_list]),
        "avg_utilization": np.mean([d["utilization"] for d in daily_stats_list]),
        "avg_queue_len": np.mean([d["avg_queue_len"] for d in daily_stats_list]),
        "avg_wait_lucky": np.mean([d["avg_wait_lucky"] for d in daily_stats_list]),
        "avg_wait_unlucky": np.mean([d["avg_wait_unlucky"] for d in daily_stats_list]),
    }
    all_replications_data.append(avg_replication_stats)

# --- RESULTS TABLE ---
print("\n\n--- Task 5: Results Summary Table (7-Day Averages) ---")

# Headers
print(f"{'Replication':<12} | {'Seed':<6} | {'Avg Util.':<10} | {'Avg Queue':<12} | {'Avg Wait (Lucky)':<22} | {'Avg Wait (Unlucky)':<24} | {'Avg Tickets':<10} | {'Avg Unlucky Cust.':<15}")
print("-" * 135)

# Data Rows
for stats in all_replications_data:
    print(f"{stats['replication']:<12} | "
          f"{stats['seed']:<6} | "
          f"{stats['avg_utilization']:<10.4f} | "
          f"{stats['avg_queue_len']:<12.4f} | "
          f"{stats['avg_wait_lucky']:<22.4f} | "
          f"{stats['avg_wait_unlucky']:<24.4f} | "
          f"{stats['avg_tickets_sold']:<10.2f} | "
          f"{stats['avg_unlucky']:<15.2f}")

# Overall Average (Average of 4 replications)
print("-" * 135)
grand_avg_util = np.mean([d["avg_utilization"] for d in all_replications_data])
grand_avg_queue = np.mean([d["avg_queue_len"] for d in all_replications_data])
grand_avg_wait_l = np.mean([d["avg_wait_lucky"] for d in all_replications_data])
grand_avg_wait_ul = np.mean([d["avg_wait_unlucky"] for d in all_replications_data])
grand_avg_tix = np.mean([d["avg_tickets_sold"] for d in all_replications_data])
grand_avg_unlucky = np.mean([d["avg_unlucky"] for d in all_replications_data])

print(f"{'GRAND AVG.':<12} | {'-':<6} | "
      f"{grand_avg_util:<10.4f} | "
      f"{grand_avg_queue:<12.4f} | "
      f"{grand_avg_wait_l:<22.4f} | "
      f"{grand_avg_wait_ul:<24.4f} | "
      f"{grand_avg_tix:<10.2f} | "
      f"{grand_avg_unlucky:<15.2f}")

--- Task 5: 7-Day Simulation (4 Replications) Run ---


--- Task 5: Results Summary Table (7-Day Averages) ---
Replication  | Seed   | Avg Util.  | Avg Queue    | Avg Wait (Lucky)       | Avg Wait (Unlucky)       | Avg Tickets | Avg Unlucky Cust.
---------------------------------------------------------------------------------------------------------------------------------------
1            | 541    | 0.4075     | 0.1093       | 0.8305                 | 0.2556                   | 138.00     | 0.14           
2            | 123    | 0.4735     | 0.2516       | 1.7344                 | 0.2682                   | 158.71     | 0.29           
3            | 789    | 0.4371     | 0.1979       | 1.4843                 | 0.2207                   | 144.71     | 0.14           
4            | 1000   | 0.3946     | 0.0890       | 0.7325                 | 0.0723                   | 122.14     | 0.14           
-------------------------------------------------------------------------------------